In [14]:
import pandas as pd
import seaborn as sns
import pylab as plt
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

df=pd.read_csv("../../../02_week/data/train.csv")

X = df[['Sex','Pclass','Age', 'Survived']]   # <=== features/independent variables
y = df['Survived']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=12)

In [2]:
Xtrain=Xtrain.reset_index()
ytrain=ytrain.reset_index()
Xtest=Xtest.reset_index()
ytest=ytest.reset_index()

In [8]:
ytest["Survived"]

0      0
1      0
2      0
3      0
4      0
      ..
174    0
175    0
176    0
177    0
178    1
Name: Survived, Length: 179, dtype: int64

In [3]:
def encode_sex(df):
    df["male"]=pd.get_dummies(df['Sex'] , drop_first=True)
    return df

def get_group_means(df):
    Group=df.groupby(["Sex","Pclass"])
    cdict={}
    meandict={}
    for (sex,pclass),sub_df in Group:
        key=sex+"_"+str(pclass)
        cdict[key]=sub_df
        meandict[key]=sub_df["Age"].mean()
    return cdict, meandict

def impose_group_means(cdict,meandict):
    for i, key in enumerate(cdict.keys()):
        cur_df=cdict[key]
        cur_df["new_Age"]=cur_df["Age"].fillna(meandict[key])

        if i==0: final_df=cur_df
        if i>0:
            final_df=pd.concat([final_df, cur_df])

    return final_df.sort_index()

In [4]:
Xtrain = encode_sex(Xtrain)
cdict_train, meandict_train = get_group_means(Xtrain)
Xtrain = impose_group_means(cdict_train, meandict_train)

In [6]:
Xtrain.tail()

,index,Sex,Pclass,Age,male,new_Age
707,241,female,3,NaN,0,21.847973
708,253,male,3,30.0,1,30.000000
709,390,male,1,36.0,1,36.000000
710,667,male,3,NaN,1,26.038824
711,843,male,3,34.5,1,34.500000


In [15]:
df=df[df["Age"].isna()==False]